In [1]:
# A logistic regression learning algorithm example using TensorFlow library.
# This example is using the MNIST database of handwritten digits 
# (http://yann.lecun.com/exdb/mnist/)

# Author: Aymeric Damien
# Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [1]:
import numpy as np
import tensorflow as tf
import read_mnist_data as rmd

# read dataset from disk
DATA_FOLDER = '/home/chengming/TensorFlow-Examples/data/'
TRAIN_IMAGES_SET = 'train-images-idx3-ubyte.gz'
TRAIN_LABELS_SET = 'train-labels-idx1-ubyte.gz'
TEST_IMAGES_SET = 't10k-images-idx3-ubyte.gz'
TEST_LABELS_SET = 't10k-labels-idx1-ubyte.gz'
validation_size = 5000
with open(DATA_FOLDER + TRAIN_IMAGES_SET, 'rb') as f:
    train_images = rmd.extract_images(f)
with open(DATA_FOLDER + TRAIN_LABELS_SET, 'rb') as f:
    train_labels = rmd.extract_labels(f, one_hot = True)
with open(DATA_FOLDER + TEST_IMAGES_SET, 'rb') as f:
    test_images = rmd.extract_images(f)
with open(DATA_FOLDER + TEST_LABELS_SET, 'rb') as f:
    test_labels = rmd.extract_labels(f, one_hot = True)
validation_images = train_images[:validation_size]
validation_labels = train_labels[:validation_size]
train_images = train_images[validation_size:]
train_labels = train_labels[validation_size:]

train_shape = train_images.shape
test_shape = test_images.shape
train_images = train_images.reshape(train_shape[0], train_shape[1]*train_shape[2])
test_images = test_images.reshape(test_shape[0], test_shape[1]*test_shape[2])


print train_images.shape
print train_labels.shape, '\t', train_labels[0][0:6], train_labels[0][6:]
print test_images.shape
print test_labels.shape, '\t', test_labels[0]

('Extracting', '/home/chengming/TensorFlow-Examples/data/train-images-idx3-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/train-labels-idx1-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/t10k-images-idx3-ubyte.gz')
('Extracting', '/home/chengming/TensorFlow-Examples/data/t10k-labels-idx1-ubyte.gz')
(55000, 784)
(55000, 10) 	[ 0.  0.  0.  0.  0.  0.] [ 0.  1.  0.  0.]
(10000, 784)
(10000, 10) 	[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


In [38]:
# Parameters
feature_size = train_images.shape[1]
label_size = train_labels.shape[1]
learning_rate = 0.01
training_epochs = 100
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, feature_size]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, label_size]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([feature_size, label_size]))
b = tf.Variable(tf.zeros([label_size]))
#W = tf.Variable(np.random.randn())
#b = tf.Variable(np.random.randn())

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
# L2 norm regularization
#L2 = tf.reduce_sum(tf.pow(W, 2)) * 0.01 / 2
#print L2.get_shape()

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred+1e-30), reduction_indices=1))
#print y*tf.log(pred+1e-10).get_shape()
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.initialize_all_variables()

In [32]:
# test
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(init)
    batch_xs = train_images[0:100, :]
    batch_ys = train_labels[0:100, :]
    _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs, y: batch_ys})
    print c
    batch_xs = train_images[100:200, :]
    batch_ys = train_labels[100:200, :]
    predict = sess.run(pred, feed_dict={x: batch_xs, y: batch_ys})
    print predict[1]-batch_ys[1]

2.30259
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [39]:
# Launch the graph
# set GPU memory usage
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_images.shape[0] / batch_size)
        start_idx = 0
        # Loop over all batches
        for i in range(total_batch):
            batch_xs = train_images[start_idx:start_idx+batch_size, :]
            batch_ys = train_labels[start_idx:start_idx+batch_size, :]
            #print start_idx
            start_idx += batch_size
            #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
            #print avg_cost
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    # tf.cast : Casts a tensor to a new type
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print "Accuracy:", accuracy.eval({x: test_images, y: test_labels})

Epoch: 0001 cost= 22.352635972
Epoch: 0002 cost= 18.554364203
Epoch: 0003 cost= 18.028639195
Epoch: 0004 cost= 17.689973224
Epoch: 0005 cost= 17.503413181
Epoch: 0006 cost= 17.520140603
Epoch: 0007 cost= 17.301704188
Epoch: 0008 cost= 17.148673163
Epoch: 0009 cost= 17.012453646
Epoch: 0010 cost= 17.005780369
Epoch: 0011 cost= 16.827179765
Epoch: 0012 cost= 16.957642365
Epoch: 0013 cost= 16.779649244
Epoch: 0014 cost= 16.757369402
Epoch: 0015 cost= 16.653903188
Epoch: 0016 cost= 16.601723987
Epoch: 0017 cost= 16.580323537
Epoch: 0018 cost= 16.604431211
Epoch: 0019 cost= 16.576754593
Epoch: 0020 cost= 16.645912450
Epoch: 0021 cost= 16.521606652
Epoch: 0022 cost= 16.642602655
Epoch: 0023 cost= 16.530296241
Epoch: 0024 cost= 16.459813909
Epoch: 0025 cost= 16.485291122
Epoch: 0026 cost= 16.460776124
Epoch: 0027 cost= 16.546569665
Epoch: 0028 cost= 16.422705550
Epoch: 0029 cost= 16.342021115
Epoch: 0030 cost= 16.377564054
Epoch: 0031 cost= 16.474805220
Epoch: 0032 cost= 16.430714701
Epoch: 0